In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Above given data is daily historical sales data of one of the largest Russian software firms - 1C Company.**

**The task is to forecast the total amount of products sold in every shop for the test set.We are trying to predict total sales for every product and store in the next month by applying time series algorithm.**

In [ ]:
df_items=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
df_items.head()

In [ ]:
df_items.shape

In [ ]:
df_item_Categories=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
df_item_Categories.head()

In [ ]:
df_item_Categories['item_category_id'].unique()

In [ ]:
df_item_Categories.shape

**it means all the id's are unique**

In [ ]:
df_sales_train=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
df_sales_train.head()

In [ ]:
df_sales_train.shape

In [ ]:
df_shops=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
df_shops.head()

In [ ]:
df_shops.shape

In [ ]:
df_test=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
df_test.head()

In [ ]:
df_test.shape

In [ ]:
#Merge the dataset df_items,df_items_categories on item_category_id
df_item=pd.merge(df_items,df_item_Categories,on='item_category_id',how='inner')
df_item.tail(12)

In [ ]:
df_item.shape

In [ ]:
# Now merge 2 dataset df_sales_train,df_shops on shop_id
df_sales_train=pd.merge(df_sales_train,df_shops,on='shop_id',how='inner')
df_sales_train.head(12)

In [ ]:
df_sales_train.shape

In [ ]:
# item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
#item_price= current price of an item
# item_price*item_cnt_day=tatal sales of particular item monthly
df_sales_train['total_Sales_monthly']=df_sales_train['item_price']*df_sales_train['item_cnt_day']
df_sales_train.head(5)

In [ ]:
#Now we will merge df_sales_train,df_item on item_id
df_sales_train=pd.merge(df_sales_train,df_item,on='item_id',how='inner')
df_sales_train.head(7)

In [ ]:
df_sales_train.shape

In [ ]:
df_sales_train.describe()

In [ ]:
df_sales_train.info()

**Date is of object data type . Convert it to DateTime data type **

In [ ]:
df_sales_train['date']=pd.to_datetime(df_sales_train['date'],format='%d.%M.%Y')
df_sales_train.head(5)

**Now the DataSet is ready to Analyse**

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from scipy.stats import boxcox
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
df_sales_train.columns

In [ ]:
#Find out the total income of each item_category in each shop
item_category_income=df_sales_train.groupby(['shop_id','item_category_id'])['total_Sales_monthly'].sum()
item_category_income.head(20)

In [ ]:
top_items=df_sales_train.groupby(['item_category_id'],as_index = False)['item_cnt_day'].count()
top_items.sort_values(by=['item_cnt_day'],ascending=False,inplace=True)
top_items.head(10)

In [ ]:
#plot it on graph
plt.figure(figsize=(20,8))
sns.barplot(data=top_items,x='item_category_id',y='item_cnt_day')
plt.xlabel('Item_Category_ID',fontsize=20)
plt.ylabel('Item_Cnt_Day',fontsize=20)
plt.title('Top Item Graph',fontsize=20)
plt.show()

**Item_category_id=40 have maximum sale**

In [ ]:
df_sales_train.date[0],df_sales_train.date[-1:]

In [ ]:
#Find out the month in which maximum sale
# date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
monthly_sales=df_sales_train.groupby(['date_block_num'],as_index=False)['item_cnt_day'].sum()

In [ ]:
plt.figure(figsize=(20,8))
sns.barplot(data=monthly_sales,x='date_block_num',y='item_cnt_day')
plt.xlabel('Month',fontsize=20)
plt.ylabel('Item_Cnt_Day',fontsize=20)
plt.title('Monthly Sales Graph',fontsize=20)
plt.show()

**11th month have maximum sale**

In [ ]:
# total Income of each shop
Shops_Monthly_income=df_sales_train.groupby(['shop_id'],as_index=False)['total_Sales_monthly'].sum()

In [ ]:
plt.figure(figsize=(20,8))
sns.barplot(data=Shops_Monthly_income,x='shop_id',y='total_Sales_monthly')
plt.xlabel('Shop ID',fontsize=20)
plt.ylabel('Income Monthly',fontsize=20)
plt.title('Monthly Sales Graph',fontsize=20)
plt.show()

In [ ]:
Shops_Monthly_income.plot(figsize=(12,4))
plt.title('Time series Plot')
plt.legend(loc='best')
plt.show()

**Shop id - 31 have maximum Income**

In [ ]:
#Let's See the income of shop_id=31 monthly wise
shop31Monthly_Income=pd.DataFrame(df_sales_train[df_sales_train['shop_id']==31])
shop31Monthly_Income.head()
shop31Monthly_Income=shop31Monthly_Income.groupby(['date_block_num'],as_index=False)['total_Sales_monthly'].sum()
shop31Monthly_Income

In [ ]:
plt.figure(figsize=(20,8))
sns.barplot(data=shop31Monthly_Income,x='date_block_num',y='total_Sales_monthly')
plt.xlabel('Months',fontsize=20)
plt.ylabel('Income Monthly',fontsize=20)
plt.title('Monthly Sales Graph of Shop 31',fontsize=20)
plt.show()

In [ ]:
shop31Monthly_Income.plot(figsize=(12, 4))
plt.legend(loc='best')
plt.title('Time Series Plot shop31')
plt.show()

**EDA part is over**

**Now coming to training of Data**

In [ ]:
#  We have total 33 months
#so, I split the data upto 25 months for training and remaining for Validation dataset
train_len=25
train_dataset=shop31Monthly_Income[:train_len]
test_dataset=shop31Monthly_Income[train_len:]

**Metrics function that i will use to evaluate the model is RMSE**

**Now try different different model for predicting the Monthly Price**

In [ ]:
#Check stationarity and seasonality of the model
from statsmodels.tsa.stattools import adfuller
adf_test = adfuller(shop31Monthly_Income['total_Sales_monthly'])

print('ADF Statistic: %f' % adf_test[0])
print('Critical Values @ 0.05: %.2f' % adf_test[4]['5%'])
print('p-value: %f' % adf_test[1])

**p-value <0.05 ,then Non-Stationary**

In [ ]:
from statsmodels.tsa.stattools import kpss
kpss_test = kpss(shop31Monthly_Income['total_Sales_monthly'])

print('KPSS Statistic: %f' % kpss_test[0])
print('Critical Values @ 0.05: %.2f' % kpss_test[3]['5%'])
print('p-value: %f' % kpss_test[1])


In [ ]:
#Now make the data stationary
# use either differencing or log transform
# Differencing removes the trend present in the series 
# while log transformation is for handling high variance in the series.

# So i will go for log transformation
train_dataset['log_total_Sales_monthly'] = pd.Series(np.log(train_dataset['total_Sales_monthly']),index=train_dataset.index)
train_dataset['log_total_Sales_monthly'] =pd.Series(train_dataset['log_total_Sales_monthly'] - train_dataset['log_total_Sales_monthly'].shift(1),index=train_dataset.index)
train_dataset['log_total_Sales_monthly'].dropna().plot()


In [ ]:
train_dataset['log_total_Sales_monthly']=train_dataset['log_total_Sales_monthly'].fillna(0)

In [ ]:
# check for stationarity
from statsmodels.tsa.stattools import adfuller
adf_test = adfuller(train_dataset['log_total_Sales_monthly'])

print('ADF Statistic: %f' % adf_test[0])
print('Critical Values @ 0.05: %.2f' % adf_test[4]['5%'])
print('p-value: %f' % adf_test[1])

**p value>0.05 - Non-Stationary**

# Boxcox transformation

In [ ]:
#Performing Box-cox transformation to get the data set stationary.
from scipy.stats import boxcox
shop_31_boxcox = pd.Series(boxcox(shop31Monthly_Income['total_Sales_monthly'], lmbda=0), index = shop31Monthly_Income.index)

plt.figure(figsize=(12,4))
plt.plot(shop_31_boxcox, label='After Box Cox tranformation')
plt.legend(loc='best')
plt.title('After Box Cox transform')
plt.show()

In [ ]:
shop31_boxcox_diff = pd.Series(shop_31_boxcox - shop_31_boxcox.shift(), shop31Monthly_Income.index)
plt.figure(figsize=(12,4))
plt.plot(shop31_boxcox_diff, label='After Box Cox tranformation and differencing')
plt.legend(loc='best')
plt.title('After Box Cox transform and differencing')
plt.show()

**All the correlation value from 1 to 24 lies beyond shaded reason ,so we will not take it into effect**

In [ ]:
shop31_boxcox_diff.dropna(inplace=True)

In [ ]:
adf_test = adfuller(shop31_boxcox_diff)

print('ADF Statistic: %f' % adf_test[0])
print('Critical Values @ 0.05: %.2f' % adf_test[4]['5%'])
print('p-value: %f' % adf_test[1])

**Now the data becomes Stationary**

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plt.figure(figsize=(8,4))
plot_acf(np.array(shop31Monthly_Income['total_Sales_monthly']),ax=plt.gca(),lags=15)
plt.show()

**Order=1 for AR model ,According to above graph**

In [ ]:
plt.figure(figsize=(8,4))
plot_pacf(shop31Monthly_Income["total_Sales_monthly"], ax=plt.gca(), lags = 15)
plt.show()

**Order =1 for Moving Average,According to above graph**

In [ ]:
shop31_boxcox_diff=pd.DataFrame(shop31_boxcox_diff,columns=['Values'])

In [ ]:
#Our stationary dataset is obtained after boxcox implementation
#  So divide the data into training and validation set
train_shop31_boxcox_diff=shop31_boxcox_diff[:train_len-1]
test_shop31_boxcox_diff=shop31_boxcox_diff[train_len-1:]

In [ ]:
test_shop31_boxcox_diff

# ARIMA model

In [ ]:
pip install pmdarima

In [ ]:

# import pmdarima as pm
from pmdarima import auto_arima
stepwise_fit=auto_arima(shop31_boxcox_diff,trace=True,supress_warnings=True)
stepwise_fit.summary()

In [ ]:
# Best model:  ARIMA(0,0,0)(0,0,0)[0] 
model=ARIMA(train_shop31_boxcox_diff['Values'],order=(1,0,1))

In [ ]:
model=model.fit()
model.summary()

In [ ]:
y_pred=shop31_boxcox_diff.copy()
y_pred['ARIMA_Pred_boxcox_diff']=model.predict(shop31_boxcox_diff.index.min(),shop31_boxcox_diff.index.max())
y_pred['ARIMA_Pred_boxCox']=y_pred['ARIMA_Pred_boxcox_diff'].cumsum()
y_pred['ARIMA_Pred_boxCox']=y_pred['ARIMA_Pred_boxCox'].add(shop_31_boxcox[0])
y_pred['ARIMA_pred']=np.exp(y_pred['ARIMA_Pred_boxCox'])

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(train_dataset['total_Sales_monthly'], label='Train')
plt.plot(test_dataset['total_Sales_monthly'], label='Test')
plt.plot(y_pred['ARIMA_pred'][test_dataset.index.min():], label='Auto regression forecast')
plt.legend(loc='best')
plt.title('Auto Regression I.M.A. Method')
plt.show()

In [ ]:
y_pred

In [ ]:
rmse = np.sqrt(mean_squared_error(test_dataset['total_Sales_monthly'], y_pred['ARIMA_pred'][test_dataset.index])).round(2)
rmse

In [ ]:
#Moving Average
model_av = ARIMA(train_shop31_boxcox_diff, order=(0, 0, 1)) 
model_fit = model_av.fit()
print(model_fit.params)

In [ ]:
y_pred=shop31_boxcox_diff.copy()
y_pred['MA_Pred_boxcox_diff']=model_fit.predict(shop31_boxcox_diff.index.min(),shop31_boxcox_diff.index.max())
y_pred['MA_Pred_boxCox']=y_pred['MA_Pred_boxcox_diff'].cumsum()
y_pred['MA_Pred_boxCox']=y_pred['MA_Pred_boxCox'].add(shop_31_boxcox[0])
y_pred['MA_pred']=np.exp(y_pred['MA_Pred_boxCox'])

In [ ]:
# y_pred
rmse = np.sqrt(mean_squared_error(test_dataset['total_Sales_monthly'], y_pred['MA_pred'][test_dataset.index])).round(2)
rmse

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(train_dataset['total_Sales_monthly'], label='Train')
plt.plot(test_dataset['total_Sales_monthly'], label='Test')
plt.plot(y_pred['MA_pred'][test_dataset.index.min():], label='MA forecast')
plt.legend(loc='best')
plt.title('M.A. Method')
plt.show()

# SARIMAX

In [ ]:
model = SARIMAX(train_shop31_boxcox_diff, order=(1, 0, 1), seasonal_order=(1, 0, 1, 15)) 
model_fit = model.fit()
print(model_fit.params)

In [ ]:
y_hat_sarima = shop31_boxcox_diff.copy()
y_hat_sarima['sarima_forecast_boxcox'] = model_fit.predict(shop31_boxcox_diff.index.min(), shop31_boxcox_diff.index.max())
y_hat_sarima['sarima_forecast'] = np.exp(y_hat_sarima['sarima_forecast_boxcox'])

In [ ]:
# y_hat_sarima
plt.figure(figsize=(12,4))
plt.plot(train_dataset['total_Sales_monthly'], label='Train')
plt.plot(test_dataset['total_Sales_monthly'], label='Test')
plt.plot(y_hat_sarima['sarima_forecast'][test_dataset.index.min():], label='SARIMA forecast')
plt.legend(loc='best')
plt.title('Seasonal autoregressive integrated moving average (SARIMA) method')
plt.show()

In [ ]:
rmse = np.sqrt(mean_squared_error(test_dataset['total_Sales_monthly'], y_hat_sarima['sarima_forecast'][test_dataset.index])).round(2)
rmse